## Imports

In [13]:
import gradio as gr # for user interface
import json # for retrieving json files from website to get price history
import urllib.parse # encoding skin names to so it can be fetched through url 
import matplotlib.pyplot as plt # visualization 
import ast  
import requests # for web scrapping
import seaborn as sns # also for visualization
import pandas as pd # data framing processed data from json so that it can be plotted
from bs4 import BeautifulSoup # for web scrapping 
import os 

## All for the LSTM model
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Make Lists

- load_skins() = Load skins from skins.txt file that was webscrapped from csgoskins.gg. This works from loading the items with format like ('Five-SeveN | Copper Galaxy', True, False) where the first boolean value is the modifier for stattrak and the second for souvenir. We also add the weapon_conditions for each weapons through a nested for loop. 
- search_list() = Makes a list of tuples to ease the searching process. 
- load_cases() = Loads cases from cases.txt file that was also webscrapped from csgoskins.gg. The logic is the same as load_skins(|)

In [14]:
weapon_conditions = [
    "(Factory New)", "(Minimal Wear)", "(Field-Tested)", "(Well-Worn)", "(Battle-Scarred)"
]

wear = [
    "Factory New", "Minimal Wear", "Field-Tested", "Well-Worn", "Battle-Scarred"
]

def load_skins():
    with open("skins.txt", "r", encoding="utf-8") as file:
        skins = [ast.literal_eval(line.strip()) for line in file.readlines()]  

    processed_skins = []
    for skin, is_stattrak, is_souvenir in skins:
        processed_skins.append(skin)  

        if is_stattrak:
            processed_skins.append(f"StatTrak {skin}")

        if is_souvenir:
            processed_skins.append(f"Souvenir {skin}")

    with_wear = []
    for skin in processed_skins:
        for conditions in weapon_conditions:
            with_wear.append(f"{skin} {conditions}")

    return with_wear

def search_list():
    with open("skins.txt", "r", encoding="utf-8") as file:
        skins = [ast.literal_eval(line.strip()) for line in file.readlines()]  

    separated_data = []
    for name, val1, val2 in skins:
        parts = name.split(" | ")
        
        if len(parts) == 2:  
            weapon, skin = parts
        else:
            weapon, skin = name, "Unknown"  
        weapon = weapon.replace("★", "")
        separated_data.append((weapon.strip(), skin.strip(), val1, val2))

    search_processed_skins = []
    for skins in separated_data:
        for w in wear:
                search_processed_skins.append((skins[0], skins[1], "Base", w))

        if skins[2]:
            for w in wear:
                search_processed_skins.append((skins[0], skins[1], "StatTrak", w))
        elif skins[3]:
            for w in wear:
                search_processed_skins.append((skins[0], skins[1], "Souvenir", w))

    return search_processed_skins

def load_cases():
    with open("case.txt", "r", encoding="utf-8") as file:
        cases = [line.strip() for line in file.readlines()]
    return cases 

## Searching Functions and URL Translation

- find_skin(sr_format, url_format, wep, s, w, m)
    - sr_format: result value from the search_list()
    - url_format: result value from the load_skin()
    - wep: name of the weapon 
    - s: name of the skin 
    - w: wear of the skin 
    - m: modifier of the skin(stattrak/souvenir)
- find_skin(): Works by matching the values given through the parameters with the list of tuples made through search_list(). Matched through their indexes. 
- find_case(c_data, name): Applies a simpler logic where it takes the data loaded from load_cases() function which is used in the c_data parameter. The name is also given to the function to match it with existing data. 
- create_url(item): Creates the link to the site containing the price history of an item in the steam market. The json will be downloaded in another phase. 

In [15]:
def find_skin(sr_format, url_format, wep,s,w,m):
    input_weapon = wep.lower()
    input_skin = s.lower() 
    input_wear = w.lower()
    input_modifier = m.lower()
    i = 0
    for items in sr_format:
        weapon = items[0].lower()
        skin = items[1].lower()
        modifier = items[2].lower()
        wear = items[3].lower()
        #print(skin,modifier,wear)
        if skin == input_skin and modifier == input_modifier and wear == input_wear and weapon == input_weapon:               
            print(f"Item Found in index : {i}")
            print(weapon, modifier, skin, wear)
            return url_format[i]
        i = i + 1
    return None 

def find_case(c_data, name):
    for cases in c_data:
        if cases.lower() == name.lower():
            return cases 

def create_url(item):
    url_format_base = "http://steamcommunity.com/market/pricehistory/?appid=730&market_hash_name="
    formatted = url_format_base+urllib.parse.quote(item, safe="★")
    return formatted.replace("%E2%98%85", "★")

## Get JSON
- get_json(url, file_name): just like its name says, it takes in the url made from create_url and gets the json from that site. It will be saved as the provided name through file_name

this also has the necessary cookies required to fetch the data

Note: For this step, it is important to have a steam account because without a steam account, you cannot fetch the price history of the item. The cookies are the ones that allows us to fetch the price history data of an item. These cookies can change once in a while so it is possible that the cookies written right now does not work. To get the cookies, you need to inspect the page where steam is logged in at and go to applications and find the sessionid and steamLoginSecure values to ensure that this part can run. Use the temporary account and replace cookie values with the logged in account. Inspect it on https://steamcommunity.com/market/ and make sure you are logged in

Temporary Steam Account: 
- Username: dia_project_2025
- Pass: Project2025

In [16]:
headers = { 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Referer": "https://steamcommunity.com",
}
# if it doesn't work, try another cookie 
cookies = { # make sure to censor in github
    "sessionid": "fill it in",
    "steamLoginSecure": "fill it in",
}

def get_json(url, file_name):
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        if response.status_code == 400:
            print(f"400 Bad Request for {url}: {response.text}") 
        response.raise_for_status()
        data = response.json()

        filename = f"{file_name}.json"  
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4)

        print(f"Saved JSON as {filename} in the root directory.")
        return True  
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return False  

# test searching
url_format = load_skins() # for skins
sr_format = search_list() # for searching
search_item = create_url(find_skin(sr_format, url_format, 'glock-18', 'water elemental', 'factory new', 'base'))
get_json(search_item, "item")

Item Found in index : 1285
glock-18 base water elemental factory new
Saved JSON as item.json in the root directory.


True

## Item Image
- This function is quite simple, you put in the name of the file and go to the listing page and then scrape the image from the site, download it to the same folder and then display it with gradio

In [17]:
def fetch_item_image(item, filename="item.jpg"):
    base_link = "https://steamcommunity.com/market/listings/730/"
    formatted = base_link + urllib.parse.quote(item, safe="★").replace("%E2%98%85", "★")

    print(formatted)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }

    response = requests.get(formatted, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    image_div = soup.find("div", class_="market_listing_largeimage")
    if not image_div:
        return "Div not found."

    img_tag = image_div.find("img", {"src": True})
    if not img_tag:
        return "Image not found."

    image_url = img_tag["src"]
    img_response = requests.get(image_url, headers=headers)

    if img_response.status_code == 200:
        os.makedirs("images", exist_ok=True)
        
        file_path = os.path.join("images", filename)
        with open(file_path, "wb") as file:
            file.write(img_response.content)
        
        return file_path  
    else:
        return "Failed to download image."

## Price Prediction

In [18]:
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [19]:
def predict_price(name):    
    try:
        with open('item.json', 'r') as f:
            data = json.load(f)
        
        dates = [entry[0] for entry in data['prices']]
        prices = [entry[1] for entry in data['prices']]

        df = pd.DataFrame({'Date': dates, 'Price': prices})
        df['Date'] = pd.to_datetime(df['Date'], format='%b %d %Y %H: +0', errors='coerce')
        df.dropna(subset=['Date'], inplace=True)
        df.set_index('Date', inplace=True)

        scaler = MinMaxScaler(feature_range=(0, 1))
        df['Scaled_Price'] = scaler.fit_transform(df[['Price']])

        train_size = len(df) - 30  # Use all data except the last month for training
        train_data = df['Scaled_Price'].iloc[:train_size].values
        test_data = df['Scaled_Price'].iloc[train_size:].values

        seq_length = 30
        X_train, y_train = create_sequences(train_data, seq_length)
        X_test, y_test = create_sequences(np.concatenate([train_data[-seq_length:], test_data]), seq_length)

        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(25),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mean_squared_error')

        model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

        predictions = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions)

        actual_prices = df['Price'].iloc[train_size:].values
        future_dates = df.index[train_size:]

        rmse = np.sqrt(mean_squared_error(actual_prices, predictions))
        print(f"RMSE: {rmse:.4f}")

        # Plot actual vs predicted values (Last 60 days: 30 before & 30 predicted)
        plt.figure(figsize=(12, 6))
        plt.plot(df.index[-60:], df['Price'][-60:], label='Actual Prices', marker='o')
        plt.plot(future_dates, predictions, label='Predicted Prices (LSTM)', linestyle='dashed', color='red', marker='o')
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'LSTM Backtesting (Actual vs Predicted - 30 Days) for {name}')
        plt.xticks(rotation=45)

        predict_path = "images/prediction.png"
        plt.savefig(predict_path)
        plt.close()

        return predict_path
        

    except Exception as e:
            print(f"{e}")

## Gradio Interface

All of the functions and how they work are implemented into two functions which are:
- process_skin_input(): Takes in user inputs and process them 
- process_case_input(): Does the same thing, just for cases 

This then is used in conjunction with the gradio interface to make it easier for users to interact with the agent.

In [ ]:
def process_skin_input(weapon_input, skin_name, condition, modifiers):
    url_format = load_skins()  # Load skin list
    sr_format = search_list()  # Load search list

    search = find_skin(sr_format, url_format, weapon_input, skin_name, condition, modifiers)
    
    if not search:
        return "Skin not found. Please check your input.", None, None, None
    
    img = fetch_item_image(search)

    url = create_url(search)
    get_json(url, "item")  

    file_path = "item.json"
    with open(file_path, "r") as file:
        data = json.load(file)

    df = pd.DataFrame(data["prices"], columns=["timestamp", "price", "volume"])
    df["price"] = df["price"].astype(float)
    df["volume"] = df["volume"].astype(int)
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%b %d %Y %H: +0", errors="coerce")
    df = df.dropna().sort_values("timestamp")

    sns.set_style("darkgrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=df["timestamp"], y=df["price"], color="b", label="Price")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(f"Price Over Time for {search}")
    plt.xticks(rotation=45)
    plt.legend()

    temp_path = "images/temp_chart.png"
    plt.savefig(temp_path)
    plt.close()

    predict_path = predict_price(search)

    return search, img, temp_path, predict_path

def process_case_input(case_name):
    case_data = load_cases()
    search = find_case(case_data, case_name)

    if not search:
        return "Case not found. Please check your input.", None, None, None

    img = fetch_item_image(search)

    url = create_url(search)
    get_json(url, "item")  

    file_path = "item.json"
    with open(file_path, "r") as file:
        data = json.load(file)

    df = pd.DataFrame(data["prices"], columns=["timestamp", "price", "volume"])
    df["price"] = df["price"].astype(float)
    df["volume"] = df["volume"].astype(int)
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%b %d %Y %H: +0", errors="coerce")
    df = df.dropna().sort_values("timestamp")

    sns.set_style("darkgrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=df["timestamp"], y=df["price"], color="b", label="Price")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(f"Price Over Time for {search}")
    plt.xticks(rotation=45)
    plt.legend()

    temp_path = "images/temp_chart.png"
    plt.savefig(temp_path)
    plt.close()

    predict_path = predict_price(search)

    return search, img, temp_path, predict_path 

with gr.Blocks() as demo:
    gr.Markdown("# Counter-Strike Market Analyzer")
    selection = gr.Radio(["Skins", "Case", "Knifes and Gloves"], label="Select Item Type [Skins/Case]", value=None)

    with gr.Column(visible=False) as skins_interface: 
        with gr.Row():
            weapon_input = gr.Dropdown(
                    ['AK-47', 'AUG', 'AWP', 'CZ75-Auto', 'Desert Eagle', 'Dual Berettas', 'FAMAS', 'Five-SeveN', 
                    'G3SG1', 'Galil AR', 'Glock-18', 'M249', 'M4A1-S', 'M4A4', 'MAG-7', 'MAC-10', 'MP5-SD', 'MP9', 
                    'Negev', 'Nova', 'P2000', 'P250', 'P90', 'PP-Bizon', 'R8 Revolver', 'SCAR-20', 'SG 553', 
                    'SSG 08', 'Sawed-Off', 'Tec-9', 'UMP-45', 'USP-S', 'XM1014', 'Zeus x27'],
                    label="Select a Weapon"
                )
            skin_name = gr.Textbox(label="Enter Skin Name: ", placeholder="e.g, Dragon Lore")
        with gr.Row():
            condition = gr.Dropdown(["Factory New", "Minimal Wear", "Field Tested", "Well Worn", "Battle-Scarred"], label="Condition")
            modifiers = gr.Dropdown([ "Base", "StatTrak", "Souvenir"], label="Pick the Modifiers")
        btn1 = gr.Button("Submit")
       
        text_msg = gr.Textbox(label="Result", interactive=False)
        item_image = gr.Image(label="Item Image")
        with gr.Row():
            priceChart = gr.Image(label="Price Trend")
            predictChart = gr.Image(label="Price Prediction")

        btn1.click(
            fn=process_skin_input,
            inputs=[weapon_input, skin_name, condition, modifiers],
            outputs=[text_msg, item_image, priceChart, predictChart]
        )

    with gr.Column(visible=False) as cases_interface:  
        case_input = gr.Dropdown(
                    [
                    "Gallery Case", "Kilowatt Case", "Revolution Case", "Recoil Case", "Dreams & Nightmares Case",
                    "Chroma 2 Case", "Chroma 3 Case", "Chroma Case", "Clutch Case", "CS20 Case", "CS:GO Weapon Case", 
                    "CS:GO Weapon Case 2", "CS:GO Weapon Case 3", "Danger Zone Case", "eSports 2013 Case", 
                    "eSports 2013 Winter Case", "eSports 2014 Summer Case", "Falchion Case", "Fracture Case", "Gamma 2 Case", 
                    "Gamma Case", "Glove Case", "Horizon Case", "Huntsman Case", "Operation Bravo Case", "Operation Breakout Weapon Case",
                    "Operation Broken Fang Case", "Operation Hydra Case", "Operation Phoenix Weapon Case", "Operation Riptide Case",
                    "Operation Vanguard Weapon Case", "Operation Wildfire Case", "Prisma 2 Case", "Prisma Case", "Revolver Case",
                    "Shadow Case", "Shattered Web Case", "Snakebite Case", "Spectrum 2 Case", "Spectrum Case", "Winter Offensive Weapon Case"
                    ],
                    label="Select a Case"
                )
        btn2 = gr.Button("Submit")
       
        text_msg = gr.Textbox(label="Result", interactive=False)
        item_image = gr.Image(label="Item Image")
        with gr.Row():
            priceChart = gr.Image(label="Price Trend")
            predictChart = gr.Image(label="Price Prediction")

        btn2.click(
            fn=process_case_input,
            inputs=[case_input],
            outputs=[text_msg, item_image, priceChart, predictChart]
        )

    with gr.Column(visible=False) as kg_interface:  
        with gr.Row():
            kg_input = gr.Dropdown(
                    [
                    "Bayonet", "Bowie Knife", "Butterfly Knife", "Classic Knife",
                    "Falchion Knife", "Flip Knife", "Gut Knife", "Huntsman Knife",
                    "Karambit", "Kukri Knife", "M9 Bayonet", "Navaja Knife", "Nomad Knife",
                    "Paracord Knife", "Shadow Daggers", "Skeleton Knife", "Stiletto Knife",
                    "Survival Knife", "Talon Knife", "Ursus Knife",
                    "Bloodhound Gloves", "Broken Fang Gloves", "Driver Gloves", "Hand Wraps",
                    "Hydra Gloves", "Moto Gloves", "Specialist Gloves", "Sport Gloves"
                    ],
                    label="Select a Weapon"
                )
            skin_name = gr.Textbox(label="Enter Skin Name: ", placeholder="e.g, Dragon Lore")
        with gr.Row():
            condition = gr.Dropdown(["Factory New", "Minimal Wear", "Field Tested", "Well Worn", "Battle-Scarred"], label="Condition")
            modifiers = gr.Dropdown(["Base","StatTrak"], label="Pick the Modifiers")
        btn3 = gr.Button("Submit")
       
        text_msg = gr.Textbox(label="Result", interactive=False)
        item_image = gr.Image(label="Item Image")
        with gr.Row():
            priceChart = gr.Image(label="Price Trend")
            predictChart = gr.Image(label="Price Prediction")

        btn3.click(
            fn=process_skin_input,
            inputs=[kg_input, skin_name, condition, modifiers],
            outputs=[text_msg, item_image, priceChart, predictChart]
        )


    def select_type(selection):
        if selection == "Skins":
            return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
        elif selection == "Case":
            return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
        elif selection == "Knifes and Gloves":
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)
        else:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)


    selection.change(
        fn=select_type,
        inputs=[selection],
        outputs=[skins_interface, cases_interface, kg_interface]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


https://steamcommunity.com/market/listings/730/Revolution%20Case
Saved JSON as item.json in the root directory.
Epoch 1/50


C:\Users\Kennard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 6.1436e-04
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 9.9990e-05
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.1251e-04
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.1813e-04
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.2023e-05
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.0267e-05
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.3499e-05
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.0016e-05
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.5290e-05
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.5705e-05
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 7.2532e-05
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.0814e-05
Epoch 13/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 4.4204e-05
Epoch 14/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.3593e-05
Epoch 15/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 